In [ ]:
import requests
import pandas as pd

Предположим, ваша задача – найти все произведения, связанные с подсолнухами. Выполните поиск по слову "sunflowers". Внимательно посмотрите на формат ответа и сохраните id полученных объектов в список sunflower_ids.  
Сам запрос к API должен состоять из нескольких параметров. Пример обращения к поиску (Search):  
r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?q=QUERY')  
res = r.json()  
где QUERY – ключевое слово.

In [2]:
r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?q=sunflower')
sunflowers = r.json()

In [3]:
sunflower_ids = sunflowers['objectIDs']

In [ ]:
sunflower_ids

Отлично! На предыдущем шаге мы получили результаты поиска. Следущий этап – собрать информацию об этих объектах.  
Пример текста запроса для получения информации об одном объекте (Object):  
r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/OBJECT_ID')  
obj_test = r_test.json()  
где OBJECT_ID – id объекта.  
Выполните запрос для объекта 437980

In [14]:
r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/437980')
obj_test = r_test.json()

In [ ]:
obj_test

Теперь напишите цикл, с помощью которого будет собрана информация об объектах, которые были получены на 2 шаге. Результаты запишите в датафрейм, а именно – следующие параметры:  
1.	 objectID – id объекта  
2.	title – название  
3.	artistDisplayName – автор  
4.	department – департамент  
5.	objectBeginDate – дата (начало)  
6.	objectEndDate – дата (конец)  
7.	period – название периода  
8.	objectName – название/категория объекта  
9.	culture – культура  


In [8]:
object_df = pd.DataFrame()
for object_id in sunflower_ids:
    r = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/{object_id}')
    obj_tmp = r.json()
    try:
        obj_tmp_df = pd.DataFrame({'objectID' : obj_tmp['objectID'],
                              'title' : [obj_tmp['title']],
                              'artistDisplayName': [obj_tmp['artistDisplayName']],
                              'department' : [obj_tmp['department']], 
                              'objectBeginDate' : [obj_tmp['objectBeginDate']], 
                              'objectEndDate' : [obj_tmp['objectEndDate']], 
                              'period' : [obj_tmp['period']], 
                              'objectName' : [obj_tmp['objectName']], 
                              'culture': [obj_tmp['culture']]
                          })
        object_df = pd.concat([object_df, obj_tmp_df])
    except KeyError:
      continue


In [16]:
object_df.head()

objectID                                title  \
0    485308                            Sunflower   
0    822590                            Sunflower   
0    707887  Botanical dish with white sunflower   
0    437112                Bouquet of Sunflowers   
0    436524                           Sunflowers   

               artistDisplayName                              department  \
0                  Joan Mitchell             Modern and Contemporary Art   
0              Brigitte Coudrain                     Drawings and Prints   
0  Chelsea Porcelain Manufactory  European Sculpture and Decorative Arts   
0                   Claude Monet                      European Paintings   
0               Vincent van Gogh                      European Paintings   

   objectBeginDate  objectEndDate period objectName culture  
0             1969           1969          Painting          
0             1950           1975             Print          
0             1750           1760              Dish          
0             1881           1881          Painting          
0             1887           1887          Painting

Сколько объектов относятся к японской культуре? (culture –Japan)

In [11]:
object_df.query("culture == 'Japan'").shape[0]

1

Теперь попробуем построить более сложный запрос с помощью секции "Search"! На этот раз параметры необходимо передать через "&", например, departmentId (id департамента – 1) и q (ключевое слово – cat):  
r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?departmentId=1&q=cat')  
cats = r.json()  
Выполните поисковый запрос для департамента "Asian art" (departmentId = 6) с тегом "cat" и запишите id объектов в cat_ids. Используйте цикл из шага 4, чтобы получить данные о первой 1000 объектов из списка.  
Note: выполнение цикла может занять некоторое время (~5 минут), поэтому проверять его работоспособность лучше на более маленькой выборке!  
В качестве ответа укажите, в каком периоде (period) было выполнено наибольшее количество работ, представленных в музее?  

In [24]:
r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?departmentId=6&q=cat')  
cats = r.json()  
cat_ids = cats['objectIDs']

In [ ]:
cat_ids

In [38]:
object_df = pd.DataFrame()
for object_id in cat_ids[:1000]:
    r = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/{object_id}')
    obj_tmp = r.json()
    try:
        obj_tmp_df = pd.DataFrame({'objectID' : obj_tmp['objectID'],
                              'title' : [obj_tmp['title']],
                              'artistDisplayName': [obj_tmp['artistDisplayName']],
                              'department' : [obj_tmp['department']], 
                              'objectBeginDate' : [obj_tmp['objectBeginDate']], 
                              'objectEndDate' : [obj_tmp['objectEndDate']], 
                              'period' : [obj_tmp['period']], 
                              'objectName' : [obj_tmp['objectName']], 
                              'culture': [obj_tmp['culture']]
                          })
        object_df = pd.concat([object_df, obj_tmp_df])
    except KeyError:
      continue

In [41]:
object_df.head()

objectID          title artistDisplayName department  objectBeginDate  \
0     49698            Cat     Zhang Yuguang  Asian Art             1900   
0     49470            Cat           Wang Li  Asian Art             1813   
0     36221            Cat          Wang Yun  Asian Art             1900   
0     53222       Musk Cat        Uto Gyoshi  Asian Art             1550   
0     60873  Head ornament                    Asian Art              960   

   objectEndDate                            period  \
0           1968                                     
0           1879          Qing dynasty (1644–1911)   
0           1933                                     
0           1599      Muromachi period (1392–1573)   
0           1279  Song dynasty (960–1279) or later   

                             objectName culture  
0  Folding fan mounted as an album leaf   China  
0                        Hanging scroll   China  
0                        Hanging scroll   China  
0                        Hanging scroll   Japan  
0                         Head ornament   China

In [42]:
object_df.period.value_counts().reset_index().iloc[0]

period    Edo period (1615–1868)
count                        312
Name: 0, dtype: object